In [1]:
import numpy as np
import json
import os.path
import random
from dataclasses import dataclass, asdict
from typing import Optional

from loguru import logger
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import SequenceClassifierOutput

logger.add("out.log")
import numpy
import requests
import torch.cuda
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import BertForSequenceClassification, AutoTokenizer, AutoModel
from sklearn.metrics import classification_report

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-5

model_path = "./Model/chinese-roberta-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = torch.load("./Model_save/classify_model.pt")

In [2]:
class DatasetClassify(Dataset):
    def __init__(self, question):
        self.data_list = [{"sentence":question,"label":0}]

    def __getitem__(self, index):
        item = self.data_list[index]
        item = DataItem(**item)
        content = item.sentence
        return content, item.label

    def __len__(self):
        return len(self.data_list)


def collator_fn(batch):
    batch = numpy.array(batch)

    data_batch = batch[:, 0]
    label_batch = numpy.array(batch[:, 1], dtype=int)
    data_batch = tokenizer(data_batch.tolist(), max_length=256, padding=True, truncation=True,
                           return_tensors="pt").to(DEVICE)
    return data_batch, torch.tensor(label_batch, device=DEVICE, dtype=torch.long)

@dataclass
class DataItem:
    sentence: str
    label: int


In [3]:
word2id = {"闲聊":0,"咨询":1,"转人工":2}
id2word = {0:"闲聊",1:"咨询",2:"转人工"}
question = "早上好"
test_data_loader = DataLoader(DatasetClassify(question), batch_size=32, shuffle=False,
                             collate_fn=collator_fn)
model.eval()
pred_label = []
for item, label in tqdm(test_data_loader, position=0, leave=True):
    model.eval()
    output = model(**item)
    pre_label = output.logits.detach().cpu().numpy()
    pre_label = np.argmax(pre_label, axis=1)
    print(id2word[int(pre_label[0])])

100%|██████████| 1/1 [00:00<00:00,  4.81it/s]

闲聊
